# Day 5: If You Give A Seed A Fertilizer

---

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

```
50 98 2
52 50 48
```

The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```

With this map, you can look up the soil number required for each initial seed number:

```
Seed number 79 corresponds to soil number 81.
Seed number 14 corresponds to soil number 14.
Seed number 55 corresponds to soil number 57.
Seed number 13 corresponds to soil number 13.
```

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

```
Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
So, the lowest location number in this example is 35.
```

What is the lowest location number that corresponds to any of the initial seed numbers?

In [40]:
import regex as re
from dataclasses import dataclass
from typing import List, Optional, Tuple

@dataclass
class Mapping:
    dest: int
    source: int
    length: int
    
    def contains(self, v: int) -> bool:
        return v >= self.source and v < self.source + self.length
    
    def map_value(self, v: int) -> int:
        return self.dest + v - self.source

    def map_range(self, r: range) -> Tuple[Optional[range], List[range]]:
        if r.start < self.source + self.length and r.stop > self.source:
            leftovers = []
            if self.contains(r.start):
                start_dest = self.map_value(r.start)
            else:
                start_dest = self.dest 
                leftovers.append(range(r.start, self.source))
            if self.contains(r.stop - 1):
                end_dest = self.map_value(r.stop - 1)
            else:
                end_dest = self.dest + self.length - 1
                leftovers.append(range(self.source + self.length, r.stop))
            return (range(start_dest, end_dest + 1), leftovers)
        else:
            return (None, [r])

seeds = []
mappings = {}
current_mapping = None
p_seed = re.compile(r"seeds:(?: +(?P<seed>[0-9]+))+")
p_mapping_name = re.compile(r"(?P<from>[a-z]+)-to-(?P<to>[a-z]+) map:")
p_mapping = re.compile(r"(?P<dest>[0-9]+) +(?P<source>[0-9]+) +(?P<length>[0-9]+)")
with open("2023_day_5_input.txt", "r") as f:
    for i, line in enumerate(f):
        if i == 0:
            seeds = [int(s) for s in p_seed.match(line).captures("seed")]
        else:
            if line == "\n":
                continue
            mapping_name_match = p_mapping_name.match(line)
            if mapping_name_match is not None:
                current_mapping = (
                    mapping_name_match.group("from"),
                    mapping_name_match.group("to")
                )
                mappings[current_mapping] = []
                continue
            mapping_match = p_mapping.match(line)
            if mapping_match is not None:
                mappings[current_mapping].append(Mapping(
                    int(mapping_match.group("dest")),
                    int(mapping_match.group("source")),
                    int(mapping_match.group("length")),
                ))
                continue

In [41]:
from typing import List

def map_seeds_to_location(seeds: List[int]):
    source = "seed"
    values = seeds.copy()
    while source != "location":
        dest = [d for s,d in mappings.keys() if s == source][0]
        for i, value in enumerate(values):
            for mapping in mappings[(source, dest)]:
                if mapping.contains(value):
                    values[i] = mapping.map_value(value)
                    break
        source = dest
    return values

min(map_seeds_to_location(seeds))

389056265

# Part Two

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [43]:
from itertools import islice

def batched(iterable, n):
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch


def map_seed_ranges_to_location_ranges(seeds: List[range]):
    source = "seed"
    ranges = seeds.copy()
    while source != "location":
        dest = [d for s,d in mappings.keys() if s == source][0]
        new_ranges = []
        for r in ranges:
            leftovers = [r]
            for mapping in mappings[(source, dest)]:
                next_leftovers = []
                for r_p in leftovers:
                    mapped_range, tmp_leftovers = mapping.map_range(r_p)
                    if mapped_range is not None:
                        new_ranges.append(mapped_range)
                    next_leftovers.extend(tmp_leftovers)
                leftovers = next_leftovers
            new_ranges.extend(leftovers)
        source = dest
        ranges = new_ranges
    return ranges

min([r.start for r in map_seed_ranges_to_location_ranges([range(s, s + l) for s, l in batched(seeds, n=2)])])

137516820